# hand label cloudy batches

In [1]:
cities = 'Dresden', 'Berlin', 'Leipzig', 'Hamburg', 'Hannover', 'Muenchen', 'Paris', 'London', 'Copenhagen', 'Erfurt'
path = "/Users/eliasstrauss/PycharmProjects/Sentinel_Building_Segmentation/"

In [16]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
for city in cities:
    img = np.load(path + "data/tensors/{}_sentinel.npy".format(city))
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(img[:, :, :3])

In [ ]:
#cut out broken part of paris
city = 'Paris'
img = np.load(path + "data/tensors/{}_sentinel.npy".format(city))
img = img[:,:3800,:]
np.save(path + "data/tensors/{}_sentinel.npy".format(city), img)

bld = np.load(path + "data/tensors/{}_building.npy".format(city))
bld = bld[:,:3800]
np.save(path + "data/tensors/{}_building.npy".format(city), bld)

In [3]:
def extract_patches_img(tensor, patch_size=(128, 128), stride=(64, 64)):
    H, W, C = tensor.shape
    patch_height, patch_width = patch_size
    stride_y, stride_x = stride

    # Calculate the shape and strides for the new view
    new_shape = (
        (H - patch_height) // stride_y + 1,
        (W - patch_width) // stride_x + 1,
        patch_height,
        patch_width,
        C
    )
    new_strides = (
        tensor.strides[0] * stride_y,
        tensor.strides[1] * stride_x,
        tensor.strides[0],
        tensor.strides[1],
        tensor.strides[2]
    )

    patches = np.lib.stride_tricks.as_strided(tensor, shape=new_shape, strides=new_strides)
    return patches

In [ ]:
np.random.seed(42)
for city in cities:
    img = np.load(path + "data/tensors/{}_sentinel.npy".format(city))
    img_patch = extract_patches_img(img)
    img_patch = img_patch.reshape((-1, 128, 128, 4))
    print(city, img_patch.shape)
    
    nr_samples = 100
    random_sample_indices = np.random.randint(0, img_patch.shape[0], nr_samples)
    samples = img_patch[random_sample_indices]
    np.save(path + "data/tensors/{}_cloud_samples.npy".format(city), samples)
    
    rows = 10  # Number of rows (you can adjust this based on your preference)
    cols = nr_samples // rows  # Calculate columns based on rows
    fig, axs = plt.subplots(rows, cols, figsize=(cols*2, rows*2))
    axs = axs.ravel()
    
    # Loop over each image and plot it
    for i in range(nr_samples):
        axs[i].imshow(samples[i,:,:,:3])
        axs[i].axis('off')  # Turn off axis labels
    
    # Remove any remaining axes that are unused
    for j in range(nr_samples, rows * cols):
        axs[j].axis('off')
    
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()
    fig.savefig(path + "data/hand_label/{}.pdf".format(city))
    

In [2]:
with open("data/hand_label/labels.txt", "r") as f:
    lines = f.readlines()

In [3]:
def extract_indices(rows):
    indices = []
    for i, row in enumerate(rows):
        offset = i*10
        index_strings = row.split(" ")
        for index_string in index_strings:
            try:
                index = int(index_string)
                indices.append(offset + index)
            except:
                continue
    return indices

In [7]:
hand_labels = {lines[i*11][:-1] : extract_indices(lines[i*11 + 1 : (i+1)*11]) for i in range(10)}

In [8]:
hand_labels.keys()

dict_keys(['Berlin', 'Copenhagen', 'Dresden', 'Erfurt', 'Hamburg', 'Hannover', 'Leipzig', 'London', 'Muenchen', 'Paris'])

In [9]:
hand_labels["Berlin"]
            

[7, 11, 13, 20, 24, 25, 38, 46, 55, 66, 69, 76, 77, 84]

In [28]:
nr_samples = 100
cloud_samples = None
cloud_labels = None
for city in cities:
    samples = np.load("data/tensors/{}_cloud_samples.npy".format(city))
    labels = np.zeros(nr_samples)
    labels[hand_labels[city]] = 1
    if cloud_samples is None:
        cloud_samples = samples
        cloud_labels = labels
    else:
        cloud_samples = np.concatenate((cloud_samples, samples))
        cloud_labels = np.concatenate((cloud_labels, labels))
    

In [30]:
np.save("data/tensors/cloud_samples.npy", cloud_samples)
np.save("data/tensors/cloud_labels.npy", cloud_labels)